# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import holoviews as hv
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dudinka,69.4058,86.1778,18.86,88,100,9.57,RU,1729452882
1,1,beaufort west,-32.3567,22.5830,52.27,74,71,15.90,ZA,1729452883
2,2,ushuaia,-54.8000,-68.3000,53.26,37,0,10.36,AR,1729452884
3,3,atherton,53.5237,-2.4935,58.28,64,55,8.99,GB,1729452885
4,4,blackmans bay,-43.0167,147.3167,52.74,93,71,4.14,AU,1729452886


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

vacation_map_plot = city_data_df.hvplot.points("Lng", 
                                          "Lat", 
                                          geo = True,
                                          color = "City",
                                          alpha = 0.75,
                                          size = "Humidity",
                                          tiles = "OSM"
                                         )

# Display the map
vacation_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
# Vacation Spot Criteria: a max temp lower than 90, higher than 80, 
# cloudiness less than 2, wind speed less than 20 mph

city_data_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 90)
                                         & (city_data_df["Max Temp"] > 80) 
                                         & (city_data_df["Cloudiness"] < 2)
                                         & (city_data_df["Wind Speed"] < 20)
                                        ]


# Drop any rows with null values
city_data_filtered_df.dropna()

# Display sample data
city_data_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
107,107,wailua homesteads,22.0669,-159.3780,81.39,77,0,11.50,US,1729453019
125,125,lihue,21.9789,-159.3672,81.99,74,0,11.50,US,1729452553
179,179,remire-montjoly,4.9167,-52.2667,87.84,70,0,18.41,GF,1729453107
205,205,umluj,25.0213,37.2685,81.45,63,0,10.98,SA,1729453138
263,263,le mars,42.7942,-96.1656,80.44,39,0,19.57,US,1729453208
273,273,gwadar,25.1216,62.3254,80.51,75,0,2.08,PK,1729453220
280,280,lompoc,34.6391,-120.4579,82.81,14,0,8.05,US,1729453228
345,345,taoudenni,22.6783,-3.9836,81.97,12,0,12.55,ML,1729453317
353,353,houma,29.5958,-90.7195,81.00,49,0,6.91,US,1729452890
371,371,ixtapa,20.7000,-105.2000,89.22,62,0,6.91,MX,1729453349


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
107,wailua homesteads,US,22.0669,-159.3780,77,
125,lihue,US,21.9789,-159.3672,74,
179,remire-montjoly,GF,4.9167,-52.2667,70,
205,umluj,SA,25.0213,37.2685,63,
263,le mars,US,42.7942,-96.1656,39,
273,gwadar,PK,25.1216,62.3254,75,
280,lompoc,US,34.6391,-120.4579,14,
345,taoudenni,ML,22.6783,-3.9836,12,
353,houma,US,29.5958,-90.7195,49,
371,ixtapa,MX,20.7000,-105.2000,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
            
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel was found!"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
lihue - nearest hotel: Kauai Palms
remire-montjoly - nearest hotel: Complexe Belova
umluj - nearest hotel: No hotel was found!
le mars - nearest hotel: Baymont LeMars
gwadar - nearest hotel: Sadaf Resorts
lompoc - nearest hotel: Red Roof Inn Lompoc
taoudenni - nearest hotel: No hotel was found!
houma - nearest hotel: Courtyard Houma
ixtapa - nearest hotel: Ma. Cristina
matoury - nearest hotel: La Marmotte
kuraymah - nearest hotel: Hotel al Nasser
dwarka - nearest hotel: The Dwarika Hotel
chikwawa - nearest hotel: Memories Lodge
maceio - nearest hotel: Maceió Hostel e Pousada


,City,Country,Lat,Lng,Humidity,Hotel Name
107,wailua homesteads,US,22.0669,-159.3780,77,Hilton Garden Inn Kauai Wailua Bay
125,lihue,US,21.9789,-159.3672,74,Kauai Palms
179,remire-montjoly,GF,4.9167,-52.2667,70,Complexe Belova
205,umluj,SA,25.0213,37.2685,63,No hotel was found!
263,le mars,US,42.7942,-96.1656,39,Baymont LeMars
273,gwadar,PK,25.1216,62.3254,75,Sadaf Resorts
280,lompoc,US,34.6391,-120.4579,14,Red Roof Inn Lompoc
345,taoudenni,ML,22.6783,-3.9836,12,No hotel was found!
353,houma,US,29.5958,-90.7195,49,Courtyard Houma
371,ixtapa,MX,20.7000,-105.2000,62,Ma. Cristina


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# remove rows for which no hotel was found because these shouldn't display in the map
hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel was found!"]

# Configure the map plot
city_hotel_map_plot = hotel_df.hvplot.points("Lng", 
                                             "Lat", 
                                             geo = True,
                                             color = "City",
                                             alpha = 0.8,
                                             width = 600,
                                             height = 400,
                                             size = "Humidity",
                                             tiles = "OSM",
                                             hover_cols = ["Hotel Name", "Country"],
                                            )

# save the map to an external file
hv.save(city_hotel_map_plot, "output_data/City_Hotel_Map.html")

# Display the map
city_hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)